# Docker

## Contenedores

### Hello world

Ejecutar el primer contenedor tipo Hello world con el comando `docker run hello-world`

In [3]:
!docker run hello-world

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world

9710123e: Pull complete 479kB/2.479kBBDigest: sha256:7d246653d0511db2a6b2e0436cfd0e52ac8c066000264b3ce63331ac66dca625
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For 

Como no tenemos el contenedor guardado en local, docker lo descarga de docker hub. Si ahora volvemos a ejecutar el contenedor, ya no aparecerá el primer mensaje, en el que indica que se está descargando

In [4]:
!docker run hello-world


Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/



Para ver los contenedores que están corriendo ejecutar `docker ps`

In [5]:
!docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


Comom vemos no hay ningún contenedor abierto. Pero sin embargo, si ejecutamos `docker ps -a` (`all`) vemos que si aparecen

In [6]:
!docker ps -a

CONTAINER ID   IMAGE         COMMAND    CREATED          STATUS                      PORTS     NAMES
fb15d590df82   hello-world   "/hello"   11 seconds ago   Exited (0) 10 seconds ago             crazy_kepler
6a15d54b8933   hello-world   "/hello"   16 seconds ago   Exited (0) 15 seconds ago             silly_swartz


Vemos que aparecen dos contenedores llamados `hello-world` que son los dos que hemos ejecutado antes. Por tanto cada vez que ejecutamos el comando `run`, docker crea un nuevo contenedor, no ejecuta uno que ya exista

Si queremos tener más información de uno de los dos contenedores podemos ejecutar `docker inspect <id>`, donde `<id>` corresponde a la ID del docker que se ha mostrado en la lista anterior

In [8]:
!docker inspect fb15d590df82

[
    {
        "Id": "fb15d590df82aa6b57287403649d3c0a8d16c82bf33225e8eff946389cc721d2",
        "Created": "2022-09-13T12:53:11.780799406Z",
        "Path": "/hello",
        "Args": [],
        "State": {
            "Status": "exited",
            "Running": false,
            "Paused": false,
            "Restarting": false,
            "OOMKilled": false,
            "Dead": false,
            "Pid": 0,
            "ExitCode": 0,
            "Error": "",
            "StartedAt": "2022-09-13T12:53:12.151172982Z",
            "FinishedAt": "2022-09-13T12:53:12.150935363Z"
        },
        "Image": "sha256:feb5d9fea6a5e9606aa995e879d862b825965ba48de054caab5ef356dc6b3412",
        "ResolvConfPath": "/var/lib/docker/containers/fb15d590df82aa6b57287403649d3c0a8d16c82bf33225e8eff946389cc721d2/resolv.conf",
        "HostnamePath": "/var/lib/docker/containers/fb15d590df82aa6b57287403649d3c0a8d16c82bf33225e8eff946389cc721d2/hostname",
        "HostsPath": "/var/lib/docker/containers/fb15

Como acordarnos de IDs es complicado para nosotros, docker asigna nombres a los contenedores para facilitarnos la vida. Así en la lista anterior, la última columna corresponde al nombre que ha asignado docker a cada contenedor, de modo que si ahora ejecutamos `docker inspect <name>` obtendremos la misma información que con la ID

Vuelvo a ejecutar `docker ps -a` para volver a ver la lista

In [9]:
!docker ps -a

CONTAINER ID   IMAGE         COMMAND    CREATED          STATUS                      PORTS     NAMES
fb15d590df82   hello-world   "/hello"   50 seconds ago   Exited (0) 49 seconds ago             crazy_kepler
6a15d54b8933   hello-world   "/hello"   55 seconds ago   Exited (0) 54 seconds ago             silly_swartz


Y ahora ejecuto `docker inspect <name>` para ver la información del contenedor

In [10]:
!docker inspect crazy_kepler

[
    {
        "Id": "fb15d590df82aa6b57287403649d3c0a8d16c82bf33225e8eff946389cc721d2",
        "Created": "2022-09-13T12:53:11.780799406Z",
        "Path": "/hello",
        "Args": [],
        "State": {
            "Status": "exited",
            "Running": false,
            "Paused": false,
            "Restarting": false,
            "OOMKilled": false,
            "Dead": false,
            "Pid": 0,
            "ExitCode": 0,
            "Error": "",
            "StartedAt": "2022-09-13T12:53:12.151172982Z",
            "FinishedAt": "2022-09-13T12:53:12.150935363Z"
        },
        "Image": "sha256:feb5d9fea6a5e9606aa995e879d862b825965ba48de054caab5ef356dc6b3412",
        "ResolvConfPath": "/var/lib/docker/containers/fb15d590df82aa6b57287403649d3c0a8d16c82bf33225e8eff946389cc721d2/resolv.conf",
        "HostnamePath": "/var/lib/docker/containers/fb15d590df82aa6b57287403649d3c0a8d16c82bf33225e8eff946389cc721d2/hostname",
        "HostsPath": "/var/lib/docker/containers/fb15

Pero por qué con `docker ps` no vemos ningún contenedor y con `docker ps -a` sí. Esto es porque `docker ps` solo muestra los contenedores que están corriendo, mientras que `docker ps -a` muestra todos los contenedores, los que están corriendo y los que están apagados

Podemos crear un contenedor asignándole un nombre nosotros mediante el comando `docker run --name <name> hello-world`

In [11]:
!docker run --name hello_world hello-world


Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/



Esto será más cómodo para nosotros, ya que podremos controlar nosotros los nombres de los contenedores

Si ahora queremos crear otro contenedor con el mismo nombre no podremos, porque docker no permite que se dupliquen los nombres de los contenedores. De modo que si queremos renombrar el contenedor podemos usar el comando `docker rename <old name> <new name>`

In [12]:
!docker rename hello_world hello_world2

Tenemos ahora un montón de contenedores iguales, pero con el mismo nombre. Así que si queremos borrar alguno tenemos que usar el comando `docker rm <id>` ó `docker rm <name>`

In [13]:
!docker ps -a

CONTAINER ID   IMAGE         COMMAND    CREATED              STATUS                          PORTS     NAMES
f503e5f0313a   hello-world   "/hello"   33 seconds ago       Exited (0) 32 seconds ago                 hello_world2
fb15d590df82   hello-world   "/hello"   About a minute ago   Exited (0) About a minute ago             crazy_kepler
6a15d54b8933   hello-world   "/hello"   About a minute ago   Exited (0) About a minute ago             silly_swartz


In [14]:
!docker rm hello_world2

hello_world2


Si volvemos a ver la lista de contenedores, el contenedor `hello_world2` ya no estará

In [16]:
!docker ps -a

CONTAINER ID   IMAGE         COMMAND    CREATED         STATUS                     PORTS     NAMES
fb15d590df82   hello-world   "/hello"   2 minutes ago   Exited (0) 2 minutes ago             crazy_kepler
6a15d54b8933   hello-world   "/hello"   2 minutes ago   Exited (0) 2 minutes ago             silly_swartz


Si queremos borrar todos los contenedores, podemos hacerlo uno a uno, pero como es muy pesado, podemos borrar todos mediante el comando `docker container prune`. Este comando elimina solo los contenedores que estén parados

In [17]:
!docker container prune

WARNING! This will remove all stopped containers.
Are you sure you want to continue? [y/N] ^C


Docker pregunta si estás seguro, y si le dices que sí, borra todos. Si ahora listo todos los contenedores no aparece ninguno

In [19]:
!docker ps -a

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


### El modo iteractivo

Vamos a ejecutar un ubuntu mediante el comando `docker run ubuntu`

In [29]:
!docker run ubuntu

Unable to find image 'ubuntu:latest' locally
latest: Pulling from library/ubuntu

Digest: sha256:20fa2d7bb4de7723f542be5923b06c4d704370f0390e4ae9e1c833c8785644c1[1A
Status: Downloaded newer image for ubuntu:latest


Como vemos ahora ha tardado más en descargar. Si listamos los contenedores mediante el comando `docker ps` vemos que no aparece el contenedor que acabamos de crear, es decir, no está corriendo

In [30]:
!docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


Listamos ahora todos los contenedores

In [31]:
!docker ps -a

CONTAINER ID   IMAGE     COMMAND   CREATED         STATUS                     PORTS     NAMES
da16b3a85178   ubuntu    "bash"    4 seconds ago   Exited (0) 3 seconds ago             hardcore_kare


Vemos que el estado del contenedor es `Exited (0)`

Si nos fijamos en comando del contenedor aparece `bash` y junto al estado `Exited (0)` nos indica que ha arrancado Ubuntu, ha ejecutado su *bash*, ha terminado la ejecución y ha devuelto un 0. Esto pasa porque al bash de Ubuntu no se le ha dicho nada que hacer. Para solucionar esto, ahora vamos a ejecutar el contenedor mediante el comando `docker run -it ubuntu`, con `it` lo que le estamos indicando es que lo queremos ejecutar en modo iterativo

```
$ docker run -it ubuntu
root@5b633e9d838f:/#
```

Ahora vemos que estamos dentro del bash de ubuntu. Si ejecutamos el comando `cat /etc/lsb-release` podemos ver la distribución de Ubuntu

```
root@5b633e9d838f:/# cat /etc/lsb-release 
DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=22.04
DISTRIB_CODENAME=jammy
DISTRIB_DESCRIPTION="Ubuntu 22.04.1 LTS"
```

Si abrimos otra terminal y vemos la lista de contenedores, ahora si aparecerá el contenedor corriendo Ubuntu

In [35]:
!docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED         STATUS         PORTS     NAMES
5b633e9d838f   ubuntu    "bash"    3 minutes ago   Up 3 minutes             funny_mirzakhani


Vemos el contenedor con Ubuntu y en su estado podemos ver `UP`

Si vemos ahora la lista de todos los contenedores, veremos que aparecen los dos contenedores con Ubuntu, el primero apagado y el segundo el que está corriendo

In [36]:
!docker ps -a

CONTAINER ID   IMAGE     COMMAND   CREATED         STATUS                     PORTS     NAMES
5b633e9d838f   ubuntu    "bash"    3 minutes ago   Up 3 minutes                         funny_mirzakhani
da16b3a85178   ubuntu    "bash"    3 minutes ago   Exited (0) 3 minutes ago             hardcore_kare


Si volvemos a la terminal donde teníamos Ubuntu corriendo dentro de un docker, si escribimos ``exit`` saldremos de Ubuntu.

```
root@5b633e9d838f:/# exit
exit
```

Si ejecutamos ``docker ps`` el contenedor ya no aparece

In [37]:
!docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


Pero si ejecuto ``docker ps -a`` sí que aparece. Esto quiere decir que el contenedor se apagó

In [38]:
!docker ps -a

CONTAINER ID   IMAGE     COMMAND   CREATED         STATUS                      PORTS     NAMES
5b633e9d838f   ubuntu    "bash"    4 minutes ago   Exited (0) 27 seconds ago             funny_mirzakhani
da16b3a85178   ubuntu    "bash"    4 minutes ago   Exited (0) 4 minutes ago              hardcore_kare


Esto ocurre porque al escribir ``exit``, en realidad lo estamos escribiendo en la consola del bash de Ubuntu, lo que signifique que estamos terminando el proceso bash de Ubuntu.

### Ciclo de vida de un contenedor

En docker, cuando el proceso principal de un contenedor se termina, se apaga el contenedor. Dentro de un contenedor pueden ejecutarse varios procesos, pero solo cuando se termina el proceso principal se apaga el contenedor

Por tanto, si queremos correr un contenedor que no se apague cuando finalice un proceso, debemos hacer que su proceso principal no se termine. En este caso, que no finalice bash

Si queremos ejecutar un contenedor con ubuntu, pero que no finalice cuando termine el proceso de bash lo podemos hacer de la siguiente manera

In [39]:
!docker run --name alwaysup -d ubuntu tail -f /dev/null

ce4d60427dcd4b326d15aa832b816c209761d6b4e067a016bb75bf9366c37054


Lo que hacemos es primero darle el nombre ``alwaysup``, en segundo lugar pasarle la opción ``-d`` (``detach``) para que el contenedor se ejecute en segundo plano y por último le decimos el proceso principal que queremos que se ejecute en el contenedor, que en este caso es ``tail -f /dev/null`` que equivale a un comando ``nop``

Esto nos devolverá la ID del contenedor, pero no estaremos dentro de ubuntu como pasaba antes

Si ahora vemos la lista de contenedores que se están ejecutando aparece el contenedor que acabamos de crear

In [40]:
!docker ps

CONTAINER ID   IMAGE     COMMAND               CREATED          STATUS          PORTS     NAMES
ce4d60427dcd   ubuntu    "tail -f /dev/null"   18 seconds ago   Up 17 seconds             alwaysup


Como ya tenemos un contenedor corriendo siempre, podemos conectarnos al el mediante el comando ``exec``. Le decimos el nombre o la ID del contenedor y le pasamos el proceso que queremos que se ejecuta. Además pasamos la opción ``-it`` para decirle que sea iteractivo

```
$ docker exec -it alwaysup bash
root@ce4d60427dcd:/#
```

Ahora volvemos a estar dentro de ubuntu. Si ejecutamos el commando ``ps -aux`` podemos ver una lista de los procesos que se están ejecutando dentro de ubuntu.

```
/# ps -aux
USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.0  0.0   2820  1048 ?        Ss   13:04   0:00 tail -f /dev/null
root           7  0.0  0.0   4628  3796 pts/0    Ss   13:04   0:00 bash
root          15  0.0  0.0   7060  1556 pts/0    R+   13:05   0:00 ps -aux
```

Vemos solo tres procesos, el ``ps -aux``, el ``bash`` y el ``tail -f /dev/null``

Este contenedor va a estar siempre encendido mientras el proceso ``tail -f /dev/null`` siga corriendo

Si salimos del contenedor con el comando ``exit`` y ejecutamos el comando ``docker ps`` vemos que el contenedor sigue encendido

```
/# exit
exit
```

In [41]:
!docker ps

CONTAINER ID   IMAGE     COMMAND               CREATED         STATUS         PORTS     NAMES
ce4d60427dcd   ubuntu    "tail -f /dev/null"   2 minutes ago   Up 2 minutes             alwaysup


Para poder finalizar el proceso y poder apagar el contenedor debemos usar el comando ``docker stop <name>``

In [53]:
!docker stop alwaysup

alwaysup


Si ahora volvemos a listar los contenedores encendidos ya no aparece el contenedor con Ubuntu

In [54]:
!docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


Y si listamos todos los contenedores, aparece el contenedor con Ubuntu, y su estado ``Exited``

In [55]:
!docker ps -a

CONTAINER ID   IMAGE     COMMAND               CREATED          STATUS                            PORTS     NAMES
ce4d60427dcd   ubuntu    "tail -f /dev/null"   14 minutes ago   Exited (137) About a minute ago             alwaysup
5b633e9d838f   ubuntu    "bash"                19 minutes ago   Exited (0) 15 minutes ago                   funny_mirzakhani
da16b3a85178   ubuntu    "bash"                20 minutes ago   Exited (0) 20 minutes ago                   hardcore_kare


### Exponer contenedores al mundo exterior

Vamos a crear un nuevo contenedor con un servidor

In [56]:
!docker run -d --name proxy nginx

Unable to find image 'nginx:latest' locally
latest: Pulling from library/nginx

f1ad4ce1: Pulling fs layer 
b079d0f8: Pulling fs layer 
5fbbebc6: Pulling fs layer 
ffdd25f4: Pulling fs layer 
32c8fba2: Pulling fs layer 
24b8ba39: Pull complete 393kB/1.393kBB[5ADigest: sha256:2888a97f7c7d498bbcc47ede1ad0f6ced07d72dfd181071dde051863f1f79d7b
Status: Downloaded newer image for nginx:latest
1a530e04f14be082811b72ea8b6ea5a95dad3037301ee8a1351a0108ff8d3b30


Esto crea un servidor, vamos a volver a listar los contenedores que están corriendo

In [57]:
!docker ps

CONTAINER ID   IMAGE     COMMAND                  CREATED        STATUS                  PORTS     NAMES
1a530e04f14b   nginx     "/docker-entrypoint.…"   1 second ago   Up Less than a second   80/tcp    proxy


Ahora aparece una nueva columna con el puerto, y nos dice que el servidor que acabamos de crear está en el puerto ``80`` bajo el protocolo ``tcp``.

Si abrimos un navegador e intentamos conectarnos al servidor mediante ``http://localhost:80`` no conseguimos conectar. Esto es porque cada contenedor tiene su propia interfaz de red. Es decir, el servidor está escuchando en el puerto ``80`` del contenedor, pero nosotros estamos intentando conectar al puerto ``80`` del host

Paramos el contenedor para relanzarlo de otra forma

In [58]:
!docker stop proxy

proxy


Si listamos los contenedores no aparece corriendo

In [59]:
!docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


Lo borramos para volver a crearlo

In [60]:
!docker rm proxy

proxy


Si listamos todos los contenedores ya no está

In [61]:
!docker ps -a

CONTAINER ID   IMAGE     COMMAND               CREATED          STATUS                       PORTS     NAMES
ce4d60427dcd   ubuntu    "tail -f /dev/null"   19 minutes ago   Exited (137) 5 minutes ago             alwaysup
5b633e9d838f   ubuntu    "bash"                24 minutes ago   Exited (0) 20 minutes ago              funny_mirzakhani
da16b3a85178   ubuntu    "bash"                24 minutes ago   Exited (0) 24 minutes ago              hardcore_kare


Para volver a crear el contenedor con el servidor y poderlo ver desde el host, tenemos que usar la opción ``-p`` (``publish``), indicando en primer lugar el puerto en el que queremos verlo en el host y a continuación el puerto del contenedor, es decir, ``-p <ip host>:<ip conteiner>``

In [62]:
!docker run -d --name proxy -p 8080:80 nginx

c199235e42f76a30266f6e1af972e0a59811806eb3d3a9afdd873f6fa1785eae


Listamos los contenedores

In [63]:
!docker ps

CONTAINER ID   IMAGE     COMMAND                  CREATED          STATUS          PORTS                                   NAMES
c199235e42f7   nginx     "/docker-entrypoint.…"   22 seconds ago   Up 21 seconds   0.0.0.0:8080->80/tcp, :::8080->80/tcp   proxy


Vemos que el puerto del contenedor es `0.0.0.0:8080->80/tcp`. Si ahora vamos a un navegador e introducimos `0.0.0.0:8080` podremos acceder al servidor del contenedor

Al listar los contenedores, en la columna ``PORTS`` indica ``0.0.0.0:8080->80/tcp``, lo que nos ayuda a ver la relación de puertos

Para ver los logs del contenedor, mediante el comando ``docker logs <name>`` puedo ver los logs del contenedor

In [64]:
!docker logs proxy

/docker-entrypoint.sh: /docker-entrypoint.d/ is not empty, will attempt to perform configuration
/docker-entrypoint.sh: Looking for shell scripts in /docker-entrypoint.d/
/docker-entrypoint.sh: Launching /docker-entrypoint.d/10-listen-on-ipv6-by-default.sh
10-listen-on-ipv6-by-default.sh: info: Getting the checksum of /etc/nginx/conf.d/default.conf
10-listen-on-ipv6-by-default.sh: info: Enabled listen on IPv6 in /etc/nginx/conf.d/default.conf
/docker-entrypoint.sh: Launching /docker-entrypoint.d/20-envsubst-on-templates.sh
/docker-entrypoint.sh: Launching /docker-entrypoint.d/30-tune-worker-processes.sh
/docker-entrypoint.sh: Configuration complete; ready for start up
2022/09/13 13:24:06 [notice] 1#1: using the "epoll" event method
2022/09/13 13:24:06 [notice] 1#1: nginx/1.23.1
2022/09/13 13:24:06 [notice] 1#1: built by gcc 10.2.1 20210110 (Debian 10.2.1-6) 
2022/09/13 13:24:06 [notice] 1#1: OS: Linux 5.15.0-46-generic
2022/09/13 13:24:06 [notice] 1#1: getrlimit(RLIMIT_NOFILE): 1048576

Ahora puedo ver todas las peticiones que se le han hecho al servidor. Pero si quiero ver los logs en tiempo real, mediante ``docker logs -f <name>`` lo puedo hacer

```
$ docker logs -f proxy
```

Ahora puedo ver los logs en tiempo real. Para salir introducir ``CTRL+C``

Como puede llegar un momento en el que haya muchos logs, si solo quieres los últimos logs, mediante la opción ``--tail <num>`` puedo ver los últimos ``<num>`` logs. Si añado la opción ``-f`` estaremos viendo siempre los últimos ``<num>`` logs

In [65]:
!docker logs --tail 10 proxy

2022/09/13 13:24:06 [notice] 1#1: start worker process 41
2022/09/13 13:24:06 [notice] 1#1: start worker process 42
172.17.0.1 - - [13/Sep/2022:13:24:16 +0000] "\x16\x03\x01\x02\x00\x01\x00\x01\xFC\x03\x03\x01E\x14\x8E\xB6\x0BEg\xF3\xC9\x9A\x19\x9C\xCA\xEC\xA7y#3\x92\x05\x95\xDCQ\x07\x19\x1D\xEF\xEA$\xBF# \x0B\x83\xF7-,s\x1B!r\xEA|\xAE\xDF\xA1\x9DLZ\xAA4y\xB3t\xAB\xC0\xCE_\xB8\xE7\xFF'\xCF\x86\x00 \xEA\xEA\x13\x01\x13\x02\x13\x03\xC0+\xC0/\xC0,\xC00\xCC\xA9\xCC\xA8\xC0\x13\xC0\x14\x00\x9C\x00\x9D\x00/\x005\x01\x00\x01\x93\x8A\x8A\x00\x00\x00\x00\x00\x0E\x00\x0C\x00\x00\x09localhost\x00\x17\x00\x00\xFF\x01\x00\x01\x00\x00" 400 157 "-" "-" "-"
172.17.0.1 - - [13/Sep/2022:13:24:16 +0000] "\x16\x03\x01\x02\x00\x01\x00\x01\xFC\x03\x03}\xA9Dr{\x8C;\x90z\x82\xAD\xBC\x8Az\xC2x\xDF\x1E\x9A\xE6l?\xA7\xE0DoK\x91'g\xBB\xB5 %\xBB\xFD\xD9\x82?\xDB\x80\xB3T\xF6cJ\xF7\xE5\xC2\xD2\x11\xBC\xA2\x1F\x90\x14\xA3\xEB\xBD=R\xBC\x83\x89\x85\x00 \xCA\xCA\x13\x01\x13\x02\x13\x03\xC0+\xC0/\xC0,\xC00\xCC\xA9\xCC\

Paramos y borramos el contenedor

In [71]:
!docker rm -f proxy

proxy


In [72]:
!docker ps -a

CONTAINER ID   IMAGE     COMMAND               CREATED          STATUS                        PORTS     NAMES
ce4d60427dcd   ubuntu    "tail -f /dev/null"   26 minutes ago   Exited (137) 13 minutes ago             alwaysup
5b633e9d838f   ubuntu    "bash"                31 minutes ago   Exited (0) 27 minutes ago               funny_mirzakhani
da16b3a85178   ubuntu    "bash"                32 minutes ago   Exited (0) 32 minutes ago               hardcore_kare


## Datos en Docker

### Bind mounts

Vamos a ver los contenedores que tenemos parados

In [73]:
!docker ps -a

CONTAINER ID   IMAGE     COMMAND               CREATED          STATUS                        PORTS     NAMES
ce4d60427dcd   ubuntu    "tail -f /dev/null"   26 minutes ago   Exited (137) 13 minutes ago             alwaysup
5b633e9d838f   ubuntu    "bash"                31 minutes ago   Exited (0) 28 minutes ago               funny_mirzakhani
da16b3a85178   ubuntu    "bash"                32 minutes ago   Exited (0) 32 minutes ago               hardcore_kare


Vamos a borrar los dos de ubuntu en los que su comando principal es la bash y vamos a dejar el que dejamos una no operación

In [75]:
!docker rm funny_mirzakhani

funny_mirzakhani


In [76]:
!docker rm hardcore_kare

hardcore_kare


In [77]:
!docker ps -a

CONTAINER ID   IMAGE     COMMAND               CREATED          STATUS                        PORTS     NAMES
ce4d60427dcd   ubuntu    "tail -f /dev/null"   27 minutes ago   Exited (137) 14 minutes ago             alwaysup


Vamos a volver a ejecutar el contenedor de ubuntu que hemos dejado, esto se hace mediante el comando `start`

In [78]:
!docker start alwaysup

alwaysup


Nos metemos otra vez dentro de el

```
$ docker exec -it alwaysup bash
root@ce4d60427dcd:/#
```

En el contenedor, puedo crear una nueva carpeta que se llame ``dockerfolder``

```
/# mkdir dockerfolder
```

Si listamos los archivos aparecerá la nueva carpeta

```
# ls
bin  boot  dev  dockerfolder  etc  home  lib  lib32  lib64  libx32  media  mnt  opt  proc  root  run  sbin  srv  sys  tmp  usr  var
```

Si salimos del contenedor

```
/# exit
exit
```

Y lo borramos

In [79]:
!docker rm -f alwaysup

alwaysup


Si listamos todos los contenedores ya no aparece el último que hemos creado

In [81]:
!docker ps -a

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


Vamos a volver a hacer todo, pero primero vamos a crear una carpeta en el host en la que compartiremos los datos con el contenedor

In [82]:
!mkdir dockerHostFolder

Vemos que dentro de la carpeta no hay nada

In [83]:
!ls dockerHostFolder

Ahora obtenemos nuestra ruta absoluta

In [84]:
!pwd

/home/wallabot/Documentos/web/portafolio/posts


Volvemos a crear el contenedor pero añadiendo la opción ``-v`` (``bind mount``). A continuación se añade la ruta absoluta de la carpeta del host y la ruta absoluta de la carpeta en el contenedor, ``-v <host path>:<container path>``

In [87]:
!docker run -d --name alwaysup -v ~/Documentos/web/portafolio/posts/dockerHostFolder:/dockerContainerFolder ubuntu tail -f /dev/null

4ede4512c293bdcc155e9c8e874dfb4a28e5163f4d5c7ddda24ad2863f28921b


Entramos al contenedor, listamos los archivos y ya aparece la carpeta que habíamos creado

```
$ docker exec -it alwaysup bash
root@4ede4512c293:/# 
root@4ede4512c293:/# ls 
bin   dev                    etc   lib    lib64   media  opt   root  sbin  sys  usr
boot  dockerContainerFolder  home  lib32  libx32  mnt    proc  run   srv   tmp  var
```

Vamos al directorio del contenedor que hemos compartido, creamos un archivo y salimos del contenedor

```
root@4ede4512c293:/# cd dockerContainerFolder/
root@4ede4512c293:/dockerContainerFolder# touch bindFile.txt
root@4ede4512c293:/dockerContainerFolder# exit
exit
```

Vemos qué hay dentro de la carpeta compartida

In [88]:
!ls dockerHostFolder

bindFile.txt


Pero es más, si borramos el contenedor, el archivo sigue ahí

In [89]:
!docker rm -f alwaysup

alwaysup


In [90]:
!ls dockerHostFolder

bindFile.txt


Si vuelvo a crear un contenedor compartiendo las carpetas, todos los archivos estarán en el contenedor

In [91]:
!docker run -d --name alwaysup -v ~/Documentos/web/portafolio/posts/dockerHostFolder:/dockerContainerFolder ubuntu tail -f /dev/null

6c021d37ea29d8b23fe5cd4968baa446085ae1756682f65340288b4c851c362d


```
$ docker exec -it alwaysup bash
root@6c021d37ea29:/# ls dockerContainerFolder/
bindFile.txt
```

Eliminamos el contenedor

In [92]:
!docker rm -f alwaysup

alwaysup


In [93]:
!docker ps -a

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


### Volúmenes

Los volúmenes se crearon como una evolución de los ``bind mounts`` para dar más seguridad. Podemos listar todos los volúmenes de docker mediante ``docker volume ls``

In [1]:
!docker volume ls

DRIVER    VOLUME NAME


Vamos a crear un nuevo volumen para el contenedor de ubuntu, para ello usamos el comando ``docker volume create <volume name>``

In [2]:
!docker volume create ubuntuVolume

ubuntuVolume


Si volvemos a listar los volúmenes aparecerá el que acabamos de crear

In [3]:
!docker volume ls

DRIVER    VOLUME NAME
local     ubuntuVolume


Sin embargo no aparece como una carpeta en el sistema de archivos del host. Con `ls -d */` listamos todas las carpetas

In [7]:
!ls -d */

dockerHostFolder/  __pycache__/


Vamos a volver a crear un contenedor, pero ahora lo creamos con el volumen que acabamos de crear con la opción ``--mount``, indicando el volumen fuente mediante ``src=<volume name>`` (si el volumen no existiese, docker lo crearía), a continuación el destino separado por una ``,``, ``dst=<container path>``, es decir ``--mount src=<volume name>,dst=<container path>``

In [8]:
!docker run -d --name alwaysup --mount src=ubuntuVolume,dst=/dockerVolumeFolder ubuntu tail -f /dev/null

42cdcddf4e46dc298a87b0570115e0b2fc900cb4c6db5eea22a61409b8cb271d


Una vez creado podemos ver los volúmenes del contenedor mediante el comando ``inspect`` y filtrando por `'{{.Mounts}}'`

```
$ docker inspect --format '{{.Mounts}}' alwaysup 
[
    {
        volume ubuntuVolume /var/lib/docker/volumes/ubuntuVolume/_data /dockerVolumeFolder local z true 
    }
]
```

Vemos que el volumen se llama `ubuntuVolume` y ademas podemos ver la ruta dónde está guardado, en este caso en `/var/lib/docker/volumes/ubuntuVolume/_data`. Hacemos lo mismo que antes, nos metemos en el contenedor, creamos un archivo en la ruta del volumen, salimos y vemos en el host si se ha creado

```
$ docker exec -it alwaysup bash
root@42cdcddf4e46:/# touch dockerVolumeFolder/volumeFile.txt
root@42cdcddf4e46:/# exit
```


```
$ sudo ls /var/lib/docker/volumes/ubuntuVolume/_data
volumeFile.txt
```

Está el archivo creado

### Insertar y extraer archivos de un contenedor

Primero vamos a crear un archivo que queremos copiar dentro de un contenedor

In [14]:
!touch dockerHostFolder/text.txt

Entramos en el contenedor

```
$ docker exec -it alwaysup bash
root@42cdcddf4e46:/#
```

Creamos una nueva carpeta donde vamos a copiar el archivo y salimos

```
root@42cdcddf4e46:/# mkdir folderToCopy
root@42cdcddf4e46:/# ls
bin  boot  dev  dockerVolumeFolder  etc  folderToCopy  home  lib  lib32  lib64  libx32  media  mnt  opt  proc  root  run  sbin  srv  sys  tmp  usr  var
root@42cdcddf4e46:/# exit
exit
```

Copiamos dentro del contenedor el archivo mediante el comando ``cp``, indicando el archivo que quiero copiar, el contenedor donde lo queremos copiar y la ruta dentro del contenedor, ``docker cp <file> <container>:<container path>``

In [15]:
!docker cp dockerHostFolder/text.txt alwaysup:/folderToCopy

Volvemos a entrar al contenedor y comprobamos que está el archivo

```
$ docker exec -it alwaysup bash
root@42cdcddf4e46:/# ls folderToCopy/
text.txt
```

Salimos del contenedor


```
/# exit
exit
```

Ahora vamos a extraer el archivo del contenedor y lo vamos a guardar en el host con otro nombre, para ello usamos el comando otra vez el comando ``cp``, pero indicando ahora el contenedor, la ruta del archivo en el contenedor y la ruta y nombre del que queremos que tenga el archivo en el host, ``docker cp <container>:<docker file path> <host file path>``

In [16]:
!docker cp alwaysup:/folderToCopy/text.txt dockerHostFolder/fileExtract.txt

Vemos que está en el host

In [17]:
!ls dockerHostFolder

bindFile.txt  fileExtract.txt  text.txt


Aunque el contenedor esté parado también se pueden copiar archivos

Por último borramos el contenedor

In [8]:
!docker rm -f alwaysup

alwaysup


## Imágenes

### Conceptos fundamentales

Las imágenes son los archivos ("plantillas") con toda la configuración para crear un contenedor. Cada vez que creamos un contenedor se crea a partir de una imagen. Cuando creábamos contenedores nuevos, la primera vez salía un mensaje diciendo que no teníamos la imagen y que iba a descargarla. En docker hub existen multitud de imágenes con todo tipo de máquinas, pero para un entorno de desarrollo muy específico podemos crear nuestra propia plantilla para pasársela a alguien y trabaje en un contenedor con la misma configuración que el nuestro

Podemos ver todas las imágenes que tenemos guardadas en nuestro ordenador mediante el comand ``docker image ls``

In [1]:
!docker image ls

REPOSITORY    TAG       IMAGE ID       CREATED         SIZE
nginx         latest    2d389e545974   8 hours ago     142MB
ubuntu        latest    2dc39ba059dc   11 days ago     77.8MB
hello-world   latest    feb5d9fea6a5   11 months ago   13.3kB


Podemos ver los tamaños, y podemos ver como la de mongodb ocupa mucho y por eso tardó más en descargarse que el resto

Otra columna que podemos ver es la de ``TAG``, esto indica la versión de la imagen. En todas pone ``latest``, esto quiere decir que es la última. Es decir, en el momento de descargárnosla nos hemos descargado la última versión que hay en docker hub. Esto en un entorno de desarrollo no es óptimo, porque nosotros podemos descacrgarnos una imagen de ubuntu, y si no especificamos versión se baja la última, por ejemplo la 20.04. Pero después de un tiempo alguien puede querer desarrollar contigo y descargarse esa imagen, pero al no especificar la versión se descargará otra vez la última, que en su caso puede ser la 22.04. Esto puede dar lugar a problemas y que cosas que a una de las personas le funcione y a la otra no

Podemos ver todas las imágenes que hay en docker hub llendo a ``https://hub.docker.com/``. Ahí podrás buscar la imagen que mejor se adapte al proyecti que quieras hacer. Si navegamos a la image de Ubuntu, por ejemplo, podemos ver las versiones (`tags`) de las imágenes.

Vamos a descargarnos, **pero no ejecutar** una imagen. Para ello usamos el comando ``docker pull <hub> <image name>:<tag>``. Si no indicamos el hub, lo descargará´de docker hub por defecto, pero podemos indicar otro, por ejemplo uno privado de nuestra organización. También, si no indicamos el tag, por defecto bajará la última versión

In [2]:
!docker pull ubuntu:20.04

20.04: Pulling from library/ubuntu

Digest: sha256:35ab2bf57814e9ff49e365efd5a5935b6915eede5c7f8581e9e1b85e0eecbe16[1A
Status: Downloaded newer image for ubuntu:20.04
docker.io/library/ubuntu:20.04


Si volvemos a listar las imágenes, vemos que ahora tenemos dos imágenes de ubuntu, una con el tag ``20.04`` y otra con el tag ``latest``

In [3]:
!docker image ls

REPOSITORY    TAG       IMAGE ID       CREATED         SIZE
nginx         latest    2d389e545974   8 hours ago     142MB
ubuntu        latest    2dc39ba059dc   11 days ago     77.8MB
ubuntu        20.04     a0ce5a295b63   11 days ago     72.8MB
hello-world   latest    feb5d9fea6a5   11 months ago   13.3kB


### Crear imágenes

Creamos un directorio en el host llamado ``images`` para trabajar en el

In [4]:
!mkdir dockerImages

Creamos un archivo ``Dockerfile`` con el que crearemos una imagen

In [5]:
!touch dockerImages/Dockerfile

Abrirmos el archivo creado con nuestro editor preferido y escribimos los siguiente:

```
FROM ubuntu:latest
```

Esto le dice a docker que cree la imagen a raiz de la imagen `latest` de ubuntu

A continuación escribimos un comando que se va a ejecutar en tiempo de compilación

```
RUN touch /test.txt
```

Esto quiere decir que cuando se compile el `Dockefile` se ejecutará ese comando, pero no cuando se corra el contenedor de la imagen

*Dockerfile*:
```dockerfile
    FROM ubuntu:latest
    RUN touch /test.txt
```

Compilamos el `Dockerfile` mediante el comando `build`, con la opción `-t` podemos darle un `tag`. Por último hay que indicarle la ruta del contexto de `build`, más adelante explicaremos esto

In [11]:
!docker build -t ubuntu:test ./dockerImages

Sending build context to Docker daemon  2.048kB
Step 1/2 : FROM ubuntu:latest
 ---> 2dc39ba059dc
Step 2/2 : RUN touch /test.txt
 ---> Using cache
 ---> a78cf3ea16d8
Successfully built a78cf3ea16d8
Successfully tagged ubuntu:test

Use 'docker scan' to run Snyk tests against images to find vulnerabilities and learn how to fix them


Como vemos se compila en 2 pasos, cada uno tiene una `id`, cada uno de esos `id`s son capas de la imagen, esto también lo veremos más adelante

Volvemos a ver las imágenes que tenemos guardadas en nuestro ordenador y aparece la que acabamos de crear

In [10]:
!docker image ls

REPOSITORY    TAG       IMAGE ID       CREATED         SIZE
ubuntu        test      a78cf3ea16d8   8 minutes ago   77.8MB
nginx         latest    2d389e545974   8 hours ago     142MB
ubuntu        latest    2dc39ba059dc   11 days ago     77.8MB
ubuntu        20.04     a0ce5a295b63   11 days ago     72.8MB
hello-world   latest    feb5d9fea6a5   11 months ago   13.3kB


Corremos el contenedor a partir de la imagen que acabamos de crear

```
$ docker run -it ubuntu:test
root@b57b9d4eedeb:/#
```

Entramos en el bash del contenedor. Como dijimos, el comando RUN se ejecutaba en tiempo de compilación de la imagen, por lo que el archivo que hemos pedido que se creara debería estar en nuestro contenedor

```
root@b57b9d4eedeb:/# ls
bin  boot  dev  etc  home  lib  lib32  lib64  libx32  media  mnt  opt  proc  root  run  sbin  srv  sys  test.txt  tmp  usr  var
```

Es importante entender que ese archivo se creó cuando se construyó la imagen, es decir, la imagen del contenedor ya tiene ese archivo. No se crea cuando se lanza el contenedor

Salimos del contenedor

```
root@b57b9d4eedeb:/# exit
exit
```

Como ya tenemos una imagen la podríamos subir al hub de docker, pero vamos a volver a listar las imágenes antes de eso

In [12]:
!docker image ls

REPOSITORY    TAG       IMAGE ID       CREATED          SIZE
ubuntu        test      a78cf3ea16d8   20 minutes ago   77.8MB
nginx         latest    2d389e545974   8 hours ago      142MB
ubuntu        latest    2dc39ba059dc   11 days ago      77.8MB
ubuntu        20.04     a0ce5a295b63   11 days ago      72.8MB
hello-world   latest    feb5d9fea6a5   11 months ago    13.3kB


Si vemos nos está diciendo que la imagen que acabamos de crear pertenece al repositorio de ubuntu, pero nosotros no tenemos acceso al repositorio de ubuntu, por lo que en docker hub nos tenemos que hacer una cuenta para poder subir la imagen a nuestro repositorio. En mi caso, mi repositorio se llama `maximofn`, por lo que cambio el repositorio de la imagen mediante el comando `tag`, indicándole la imagen a la que queremos cambiar de repositorio y el nuevo repositorio. En el nuevo repositorio se suele indicar el nombre del repositorio seguido del tipo de imagen y el tag, en mi caso `maximofn/ubuntu:test`

In [13]:
!docker tag ubuntu:test maximofn/ubuntu:test

Si ahora volvemos a listar las imágenes

In [14]:
!docker image ls

REPOSITORY        TAG       IMAGE ID       CREATED          SIZE
ubuntu            test      a78cf3ea16d8   24 minutes ago   77.8MB
maximofn/ubuntu   test      a78cf3ea16d8   24 minutes ago   77.8MB
nginx             latest    2d389e545974   8 hours ago      142MB
ubuntu            latest    2dc39ba059dc   11 days ago      77.8MB
ubuntu            20.04     a0ce5a295b63   11 days ago      72.8MB
hello-world       latest    feb5d9fea6a5   11 months ago    13.3kB


Ahora debemos loggearnos dentro de docker hub para poder subir la imagen, para ello usamos el comando `login`

```
$ docker login
Login with your Docker ID to push and pull images from Docker Hub. If you don't have a Docker ID, head over to https://hub.docker.com to create one.
Username: maximofn
Password:

Login Succeeded
```

Ahora podemos subir la imagen mediante el comando `pull`

In [16]:
!docker push maximofn/ubuntu:test

The push refers to repository [docker.io/maximofn/ubuntu]

06994357: Preparing 
06994357: Pushed  from library/ubuntu test: digest: sha256:318d83fc3c35ff930d695b0dc1c5ad1b0ea54e1ec6e3478b8ca85c05fd793c4e size: 735


Ha subido solo la primera capa, la segunda, como la use a raiz de la imagen de ubuntu, lo que hace es colocar un puntero a esa imagen para no tener capas subidas más de una vez

Hay que tener en cuenta que este repositorio es público, por lo que no debes subir imágenes con datos sensibles. Además, si una imagen no tiene uso en 6 meses será borrada

### El sistema de capas

Mediante el comando `history` podemos ver las capas de una imagen. Si vemos las capas de la imagen que acabamos de crear usamos `docker history ubuntu:test`

In [1]:
!docker history ubuntu:test

IMAGE          CREATED        CREATED BY                                      SIZE      COMMENT
a78cf3ea16d8   16 hours ago   /bin/sh -c touch /test.txt                      0B        
2dc39ba059dc   12 days ago    /bin/sh -c #(nop)  CMD ["bash"]                 0B        
<missing>      12 days ago    /bin/sh -c #(nop) ADD file:a7268f82a86219801…   77.8MB    


Vemos que la primera capa tiene el comando que hemos introducido en el `Dockerfile`, además dice que ha sido creada hace 16 horas (me fui a dormir desde el anterior comando a este :-) ). Sin embargo, el resto de capas fueron creadas hace 12 días, y son las capas de la imagen de ubuntu de la que nos hemos basado

Al `Dockerfile` que hemos creado antes le añadimos la línea 

```
RUN rm /test.txt
```

*Dockerfile*:
```dockerfile
    FROM ubuntu:latest
    RUN touch /test.txt
    RUN rm /test.txt
```

Si volvemos a compilar, vemos qué pasa

In [2]:
!docker build -t ubuntu:test ./dockerImages

Sending build context to Docker daemon  2.048kB
Step 1/3 : FROM ubuntu:latest
 ---> 2dc39ba059dc
Step 2/3 : RUN touch /test.txt
 ---> Using cache
 ---> a78cf3ea16d8
Step 3/3 : RUN rm /test.txt
 ---> Running in c2e6887f2025
Removing intermediate container c2e6887f2025
 ---> 313243a9b573
Successfully built 313243a9b573
Successfully tagged ubuntu:test

Use 'docker scan' to run Snyk tests against images to find vulnerabilities and learn how to fix them


Como vemos hay una capa más con la nueva línea que hemos agregado. Si volvemos a ver las capas de la imagen con `history`

In [3]:
!docker history ubuntu:test

IMAGE          CREATED              CREATED BY                                      SIZE      COMMENT
313243a9b573   About a minute ago   /bin/sh -c rm /test.txt                         0B        
a78cf3ea16d8   16 hours ago         /bin/sh -c touch /test.txt                      0B        
2dc39ba059dc   12 days ago          /bin/sh -c #(nop)  CMD ["bash"]                 0B        
<missing>      12 days ago          /bin/sh -c #(nop) ADD file:a7268f82a86219801…   77.8MB    


Vemos que las primeras capas son iguales que antes y ha añadido una nueva capa con el nuevo comando

## Uso de docker para crear aplicaciones

### Exposición de puertos

Antes vimos como podíamos vincular un puerto de un contenedor a un puerto del ordenador (`-p 8080:80`). Pero para que eso sea posible, a la hora de crear la imagen hay que exponer el puerto, esto se hace añadiendo al Dockerfile la linea `EXPOSE <port>`, en el caso de antes

```
EXPOSE 80
```

O usar imágenes como base que ya tengan puertos expuestos

### Reuso del caché de capas al compilar

Cuando compilamos una imagen, si alguna de las capas que hemos definido ya han sido ocmpiladas antes, docker lo detecta y las usa, no las vuelve a compilar. Si volvemos a compilar la imagen que hemos definido en el `Dockerfile` ahora tardará muy poco, porque todas las capas ya están compiladas y docker no las vuelve a compilar

In [4]:
!docker build -t ubuntu:test ./dockerImages

Sending build context to Docker daemon  2.048kB
Step 1/3 : FROM ubuntu:latest
 ---> 2dc39ba059dc
Step 2/3 : RUN touch /test.txt
 ---> Using cache
 ---> a78cf3ea16d8
Step 3/3 : RUN rm /test.txt
 ---> Using cache
 ---> 313243a9b573
Successfully built 313243a9b573
Successfully tagged ubuntu:test

Use 'docker scan' to run Snyk tests against images to find vulnerabilities and learn how to fix them


Como esto es un cuaderno Jupyter a la hora de ejecutar las celdas te da la información del tiempo que tardan en ejecutarse, la vez anterior que compilé la imagen (en realidad solo compilé la última capa que cree) tardó 1,4 segundos, mientras que ahora ha tardado 0,5 segundos

Pero si ahora cambio el Dockerfile, y en la primera línea, donde decía que nos basábamos en la última versión de ubuntu y cambiamos a la versión 20.04

```
FROM ubuntu:20.04
```

*Dockerfile*:
```dockerfile
    FROM ubuntu:20.04
    RUN touch /test.txt
    RUN rm /test.txt
```

Si volvemos a compilar tardará mucho más

In [5]:
!docker build -t ubuntu:test ./dockerImages

Sending build context to Docker daemon  2.048kB
Step 1/3 : FROM ubuntu:20.04
 ---> a0ce5a295b63
Step 2/3 : RUN touch /test.txt
 ---> Running in a40fe8df2c0d
Removing intermediate container a40fe8df2c0d
 ---> 0bb9b452c11f
Step 3/3 : RUN rm /test.txt
 ---> Running in 2e14919f3685
Removing intermediate container 2e14919f3685
 ---> fdc248fa833b
Successfully built fdc248fa833b
Successfully tagged ubuntu:test

Use 'docker scan' to run Snyk tests against images to find vulnerabilities and learn how to fix them


Ha tardado 1,9 segundos

Al cambiar la primera capa, docker vuelve a compilar todas las capas. Esto puede ser un problema porque a la hora de desarrollar código se puede dar el siguiente caso
 * Desarrollamos el código en nuestro ordenador
 * Al construir la imagen copiamos todo el código de nuestro ordenador al contenedor
 * Luego le pedimos a la imagen que instale las librerías necesarias

Esto puede hacer que al cambiar cualquier parte del código, al tener que volver a compilar la imagen, la capa en la que se instalan las librerías se tenga que volver a compilar, ya que ha cambiado una capa anterior

Para solucionar esto la idea sería que a la hora de crear la imagen, primero pidamos que se instalen las librerías y luego que se copie el código de nuestro ordenador al contenedor. Así cada vez que cambiemos el código y volvamos a compilar la imagen, solo se recompilará la capa en la que se copia el código, por lo que la compilación será más rápida

Podrás pensar, que es mejor compartir una carpeta entre el host y el contenedor (`bind mount`) donde tendremos el código y así no hace falta volver a compilar la imagen cada vez que cambiemo el código. Y la respuesta es que es verdad, solo he puesto este ejemplo porque es muy fácil de entender, pero es para escenificar que a la hora de crear imágenes hay que pensar bien de manera que si hace falta volver a compilarla, recompile el mínimo número de capas

### Código compartido en una carpeta `bind mount`

Habíamos creado una carpeta llamada `dockerHostFolder` en la que habíamos compartido archivos entre el host y un contenedor. Dentro además debería haber tres archivos

In [1]:
!ls dockerHostFolder

bindFile.txt  fileExtract.txt  text.txt


Vamos a aprovechar el archivo `text.txt` para ver eso. Vamos a ver qué hay dentro de `text.txt`

In [3]:
!cat dockerHostFolder/text.txt

No hay salida, el archivo está vacío. Vamos a crear otra vez un contenedor de ubuntu compartiendo la carpeta `dockerHostFolder`

In [4]:
!docker run --name alwaysup -d -v ~/Documentos/web/portafolio/posts/dockerHostFolder:/dockerContainerFolder ubuntu tail -f /dev/null

24adbded61f507cdf7f192eb5e246e43ee3ffafc9944b7c57918eb2d547dff19


Vemos que el contenedor está corriendo

In [5]:
!docker ps

CONTAINER ID   IMAGE     COMMAND               CREATED          STATUS          PORTS     NAMES
24adbded61f5   ubuntu    "tail -f /dev/null"   16 seconds ago   Up 15 seconds             alwaysup


Entramos en el contenedor, vemos que está `text.txt` y que está vacío

```
$ docker exec -it alwaysup bash
root@24adbded61f5:/# ls dockerContainerFolder/
bindFile.txt  fileExtract.txt  text.txt
root@24adbded61f5:/# cat dockerContainerFolder/text.txt 
root@24adbded61f5:/# 
```

Ahora abrimos en el host el archivo `text.txt` con el editor de textos que queramos, escribimos `Hola mundo` y guardamos. Si ahora vemos qué hay dentro del archivo en el contenedor veremos el mismo texto

```
root@24adbded61f5:/# cat dockerContainerFolder/text.txt 
Hola mundo
```

Ahora editamos el archivo en el contenedor y salimos del contenedor

```
root@24adbded61f5:/# echo hola contenedor > dockerContainerFolder/text.txt 
root@24adbded61f5:/# cat dockerContainerFolder/text.txt 
hola contenedor
root@24adbded61f5:/# exit
exit
```

Si miramos el archivo en el host veremos el texto que escribimos en el contenedor

In [6]:
!cat dockerHostFolder/text.txt

hola contenedor


Borramos el contenedor

In [7]:
!docker rm -f alwaysup

alwaysup


### Conectar contenedores por red

En caso de que queramos tener varios contenedores corriendo y queremos que se comuniquen, podemos hacer que se comuniquen por red. Docker nos da la posibilidad de hacer eso mediante sus redes virtuales

Vamos a ver qué redes tiene docker mediante el comando `docker network ls`

In [8]:
!docker network ls

NETWORK ID     NAME      DRIVER    SCOPE
de6e8b7b737e   bridge    bridge    local
da1f5f6fccc0   host      host      local
d3b0d93993c0   none      null      local


Vemos que por defecto docker tiene tres redes
 * bridge: Está por retrocompatibilidad con versiones anteriores pero no deberíamos usarla ya
 * host: Es la red del host
 * none: Esta es la que debemos usar si queremos que un contenedor no tenga acceso a internet

Podemos crear redes nuevas que otros contenedores se puedan conectar a ella, para ello usamos el comando `docker network create <name>`, para que otros contenedores se puedan conectar además debemos añadir la opción `--attachable`

In [11]:
!docker network create --attachable myNetwork

2f6f3ddbfa8642e9f6819aa0965c16339e9e910be7bcf56ebb718fcac324cc27


Podemos inspeccionarla mediante el comando `docker network inspect <name>`

In [12]:
!docker network inspect myNetwork

[
    {
        "Name": "myNetwork",
        "Id": "2f6f3ddbfa8642e9f6819aa0965c16339e9e910be7bcf56ebb718fcac324cc27",
        "Created": "2022-09-14T15:20:08.539830161+02:00",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": {},
            "Config": [
                {
                    "Subnet": "172.18.0.0/16",
                    "Gateway": "172.18.0.1"
                }
            ]
        },
        "Internal": false,
        "Attachable": true,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {},
        "Options": {},
        "Labels": {}
    }
]


Ahora tenemos que crear dos contenedores para que se puedan comunicar.

Vamos a crear un nuevo contenedor, que llamaremos `container1`, con una carpeta compartida y que en su interior se va a llamar `folder1`

In [17]:
!docker run --name container1 -d -v ~/Documentos/web/portafolio/posts/dockerHostFolder:/folder1 ubuntu tail -f /dev/null

a5fca8ba1e4ff0a67002f8f1b8cc3cd43185373c2a7e295546f774059ad8dd1a


Ahora creamos otro contenedor, llamado `container2`, con otra carpeta compartida, pero que se llame `folder2`

In [18]:
!docker run --name container2 -d -v ~/Documentos/web/portafolio/posts/dockerHostFolder:/folder2 ubuntu tail -f /dev/null

6c8dc18315488ef686f7548516c19b3d716728dd8a173cdb889ec0dd082232f9


Vemos los contenedores corriendo y vemos que stán los dos

In [19]:
!docker ps

CONTAINER ID   IMAGE     COMMAND               CREATED         STATUS         PORTS     NAMES
6c8dc1831548   ubuntu    "tail -f /dev/null"   3 seconds ago   Up 2 seconds             container2
a5fca8ba1e4f   ubuntu    "tail -f /dev/null"   4 seconds ago   Up 3 seconds             container1


Ahora tenemos que conectar los contenedores a la red, para ello usamos el comando `docker network connect <network name> <container name>`

In [20]:
!docker network connect myNetwork container1

In [21]:
!docker network connect myNetwork container2

Para comprobar que se han conectado bien podemos inspeccionar la red, pero filtrando por los contenedores conectados

```
$ docker network inspect --format '{{.Containers}}' myNetwork 
map
[
    6c8dc18315488ef686f7548516c19b3d716728dd8a173cdb889ec0dd082232f9:
    {
        container2 
        f828d211e894f7a5a992ce41a2a0def8e2424e9737fb4e1485fc09cc2d607b69 
        02:42:ac:12:00:03 
        172.18.0.3/16 
    }
    a5fca8ba1e4ff0a67002f8f1b8cc3cd43185373c2a7e295546f774059ad8dd1a:
    {
        container1 
        cff762e6286ebc169804b2a675bbff904102de796751d367c18d4b490c994c45 
        02:42:ac:12:00:02 
        172.18.0.2/16 
    }
]
```

Como podemos ver el contenedor `container1` tiene la IP `172.18.0.2` y el contenedor `container2` tiene la IP `172.18.0.3`

Nos metemos dentro del contenedor `container1` e instalamos `ping`

```
$ docker exec -it container1 bash
root@a5fca8ba1e4f:/# apt update
    ...
root@a5fca8ba1e4f:/# apt install iputils-ping
    ...
root@a5fca8ba1e4f:/#
```

Nos metemos dentro del contenedor `container2` e instalamos `ping`

```
$ docker exec -it container2 bash
root@a5fca8ba1e4f:/# apt update
    ...
root@a5fca8ba1e4f:/# apt install iputils-ping
    ...
root@a5fca8ba1e4f:/#
```

Ahora desde el contenedor `container1` hacemos un ping a la IP `172.18.0.3`, que pertenece al contenedor `container2`

```
root@a5fca8ba1e4f:/# ping 172.18.0.3
PING 172.18.0.3 (172.18.0.3) 56(84) bytes of data.
64 bytes from 172.18.0.3: icmp_seq=1 ttl=64 time=0.115 ms
64 bytes from 172.18.0.3: icmp_seq=2 ttl=64 time=0.049 ms
64 bytes from 172.18.0.3: icmp_seq=3 ttl=64 time=0.056 ms
64 bytes from 172.18.0.3: icmp_seq=4 ttl=64 time=0.060 ms
^C
--- 172.18.0.3 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3068ms
rtt min/avg/max/mdev = 0.049/0.070/0.115/0.026 ms
```

Y desde el contenedor `container2` hacemos un ping a la IP `172.18.0.2`, que pertenece al contenedor `container1`

```
root@6c8dc1831548:/# ping 172.18.0.2
PING 172.18.0.2 (172.18.0.2) 56(84) bytes of data.
64 bytes from 172.18.0.2: icmp_seq=1 ttl=64 time=0.076 ms
64 bytes from 172.18.0.2: icmp_seq=2 ttl=64 time=0.045 ms
64 bytes from 172.18.0.2: icmp_seq=3 ttl=64 time=0.049 ms
64 bytes from 172.18.0.2: icmp_seq=4 ttl=64 time=0.051 ms
^C
--- 172.18.0.2 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3074ms
rtt min/avg/max/mdev = 0.045/0.055/0.076/0.012 ms
```

Pero hay una cosa mejor que nos permite hacer docker, si no me sé la IP del contenedor al que me quiero conectar, en vez de escribir su IP puedo escribir su nombre

Ahora desde el contenedor `container1` hacemos un ping a la IP `container2`

```
root@a5fca8ba1e4f:/# ping container2
PING container2 (172.18.0.3) 56(84) bytes of data.
64 bytes from container2.myNetwork (172.18.0.3): icmp_seq=1 ttl=64 time=0.048 ms
64 bytes from container2.myNetwork (172.18.0.3): icmp_seq=2 ttl=64 time=0.050 ms
64 bytes from container2.myNetwork (172.18.0.3): icmp_seq=3 ttl=64 time=0.052 ms
64 bytes from container2.myNetwork (172.18.0.3): icmp_seq=4 ttl=64 time=0.053 ms
^C
--- container2 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3071ms
rtt min/avg/max/mdev = 0.048/0.050/0.053/0.002 ms
```

Como vemos docker sabe que la IP del contenedor `conteiner2` es la `172.18.0.3`

Y desde el contenedor `container2` hacemos un ping a la IP `container1`

```
root@6c8dc1831548:/# ping container1
PING container1 (172.18.0.2) 56(84) bytes of data.
64 bytes from container1.myNetwork (172.18.0.2): icmp_seq=1 ttl=64 time=0.051 ms
64 bytes from container1.myNetwork (172.18.0.2): icmp_seq=2 ttl=64 time=0.058 ms
64 bytes from container1.myNetwork (172.18.0.2): icmp_seq=3 ttl=64 time=0.052 ms
64 bytes from container1.myNetwork (172.18.0.2): icmp_seq=4 ttl=64 time=0.056 ms
^C
--- container1 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3057ms
rtt min/avg/max/mdev = 0.051/0.054/0.058/0.003 ms
```

Como vemos docker sabe que la IP del contenedor `conteiner1` es la `172.18.0.2`

Salimos de los contenedores y los borramos

In [23]:
!docker rm -f container1 container2

container1
container2


Borramos también la red que hemos creado

In [25]:
!docker network rm myNetwork

myNetwork


## Docker compose

Docker compose es una herramienta de docker que hace todo lo que hemos visto hasta ahora, pero ahorrándonos tiempo y esfuerzo. Editando un archivo `.yml` podemos decirle a docker compose que cree todos los contenedores que queramos.

Para usarlo una vez no habrá mucha diferencia de escribir todos los comandos que vimos antes o escribir el archivo `.yml` pero cuando quires volver a tener la misma configuración de contenedores trabajando, solo con llamar al archivo `.yml` volverá a crear toda la configuración

Vamos a crear una carpeta donde guardaremos los archivos de docker compose

In [26]:
!mkdir dockerComposeFiles

Creamos dentro el archivo `.yml`

In [27]:
!touch dockerComposeFiles/docker-compose.yml

Un archivo docker compose tiene que empezar por la versión

```yml
version: "<v.v>"
```

En el momento de escribir esto, la última versión es la `3.8` así que escribimos esa

*docker-compose.yml*:

```yml
    version: "3.8"
```

A continuación se indican los servicios, que son los contenedores, en cada servicio hay que especificar la imagen y además se pueden añadir otros parámetros como puertos, variables de entorno, etc

```yml
services:
    container1:
        image: ubuntu
    
    container2:
        image: ubuntu
```

*docker-compose.yml*:

```yml
    version: "3.8"

    services:
    container1:
        image: ubuntu
    
    container2:
        image: ubuntu
```

Una vez que hemos creado el archivo, en su path, podemos ejecutar todo mediante el comando `docker compose up`, pero además añadiendo la opción `-d` haremos que corra en segundo plano

In [30]:
!cd dockerComposeFiles && docker compose up -d

[+] Running 1/0
 ⠿ Network dockercomposefiles_default         Created                      0.1s
 ⠋ Container dockercomposefiles-container2-1  Creating                     0.0s
 ⠋ Container dockercomposefiles-container1-1  Creating                     0.0s
[+] Running 1/3
 ⠿ Network dockercomposefiles_default         Created                      0.1s
 ⠙ Container dockercomposefiles-container2-1  Creating                     0.1s
 ⠙ Container dockercomposefiles-container1-1  Creating                     0.1s
[+] Running 1/3
 ⠿ Network dockercomposefiles_default         Created                      0.1s
 ⠿ Container dockercomposefiles-container2-1  Starting                     0.2s
 ⠿ Container dockercomposefiles-container1-1  Starting                     0.2s
[+] Running 1/3
 ⠿ Network dockercomposefiles_default         Created                      0.1s
 ⠿ Container dockercomposefiles-container2-1  Starting                     0.3s
 ⠿ Container dockercomposefiles-container1-1  Starting  

Si nos fijamos ha creado dos contenedores `dockercomposefiles-container1-1` y `dockercomposefiles-container2-1` y la red que los une `dockercomposefiles_default`

Vamos a borrar los dos contenedores

In [34]:
!docker rm -f dockercomposefiles-container1-1 dockercomposefiles-container2-1

dockercomposefiles-container1-1
dockercomposefiles-container2-1


Y borramos la red que ha creado

In [35]:
!docker network rm dockercomposefiles_default

dockercomposefiles_default
